**IMPORTING LIBRARIES**

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense  
from google.colab import drive

**CONNECTING TO OUR DRIVE**

In [16]:
drive.mount("/content/drive")
dataframe=pd.read_csv(r'/content/drive/MyDrive/House Price India.csv')
dataframe.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,Date,number of bedrooms,number of bathrooms,living area,lot area,number of floors,waterfront present,number of views,condition of the house,...,Built Year,Renovation Year,Postal Code,Lattitude,Longitude,living_area_renov,lot_area_renov,Number of schools nearby,Distance from the airport,Price
0,6762810145,42491,5,2.50,3650,9050,2.0,0,4,5,...,1921,0,122003,52.8645,-114.557,2880,5400,2,58,2380000
1,6762810635,42491,4,2.50,2920,4000,1.5,0,0,5,...,1909,0,122004,52.8878,-114.470,2470,4000,2,51,1400000
2,6762810998,42491,5,2.75,2910,9480,1.5,0,0,3,...,1939,0,122004,52.8852,-114.468,2940,6600,1,53,1200000
3,6762812605,42491,4,2.50,3310,42998,2.0,0,0,3,...,2001,0,122005,52.9532,-114.321,3350,42847,3,76,838000
4,6762812919,42491,3,2.00,2710,4500,1.5,0,0,4,...,1929,0,122006,52.9047,-114.485,2060,4500,1,51,805000


**DATA PREPROCESSING**

In [17]:
dataframe_modified=dataframe.drop(["Date","id","Lattitude","Longitude"],axis=1)

dataframe_modified["number of bathrooms"]=dataframe["number of bathrooms"].round().astype(int)
dataframe_modified["number of floors"]=dataframe["number of floors"].round().astype(int)
dataframe_modified["PriceRange"]=pd.cut(dataframe["Price"],bins=[0, 500000, 1000000, 1500000, float('inf')],labels=["0-500k", "500k-1M", "1M-1.5M", "1.5M+"])
Count=dataframe_modified["PriceRange"].value_counts()
dataframe_modified["Count"] = dataframe_modified["PriceRange"].map(Count)
dataframe_modified.head(40)

,number of bedrooms,number of bathrooms,living area,lot area,number of floors,waterfront present,number of views,condition of the house,grade of the house,Area of the house(excluding basement),...,Built Year,Renovation Year,Postal Code,living_area_renov,lot_area_renov,Number of schools nearby,Distance from the airport,Price,PriceRange,Count
0,5,2,3650,9050,2,0,4,5,10,3370,...,1921,0,122003,2880,5400,2,58,2380000,1.5M+,333
1,4,2,2920,4000,2,0,0,5,8,1910,...,1909,0,122004,2470,4000,2,51,1400000,1M-1.5M,646
2,5,3,2910,9480,2,0,0,3,8,2910,...,1939,0,122004,2940,6600,1,53,1200000,1M-1.5M,646
3,4,2,3310,42998,2,0,0,3,9,3310,...,2001,0,122005,3350,42847,3,76,838000,500k-1M,5122
4,3,2,2710,4500,2,0,0,4,8,1880,...,1929,0,122006,2060,4500,1,51,805000,500k-1M,5122
5,3,2,2600,4750,1,0,0,4,9,1700,...,1951,0,122007,2380,4750,1,67,790000,500k-1M,5122
6,5,3,3660,11995,2,0,2,3,10,3660,...,2006,0,122008,3320,11241,3,72,785000,500k-1M,5122
7,3,2,2240,10578,2,0,0,5,8,1550,...,1923,0,122006,1570,10578,3,71,750000,500k-1M,5122
8,3,2,2390,6550,1,0,2,4,8,1440,...,1955,0,122009,2010,6550,1,73,750000,500k-1M,5122
9,4,2,2200,11250,2,0,0,5,7,1300,...,1920,0,122010,2320,10814,2,53,698000,500k-1M,5122


In [18]:
dataframe_modified=dataframe_modified.drop(["Count","Price"],axis=1)
print(dataframe_modified.shape)
dataframe_modified.isnull().sum()
dataframe_modified.shape

(14620, 19)


(14620, 19)

In [8]:
label_map = {'0-500k': 0, '500k-1M': 1, '1M-1.5M': 2, '1.5M+': 3}
label_encoder = LabelEncoder()
dataframe_modified["PriceRangeEncoded"] = label_encoder.fit_transform(dataframe_modified['PriceRange'].apply(lambda x: label_map[x]))

y = dataframe_modified["PriceRangeEncoded"].values

X = dataframe_modified.drop(columns=['PriceRange','PriceRangeEncoded'],axis = 1).values

y_o_h = tf.keras.utils.to_categorical(y,num_classes = 4)
X_train, X_test, y_train, y_test = train_test_split(X, y_o_h, test_size=0.3, random_state=42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='swish', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(64, activation='swish'),
    tf.keras.layers.Dense(32, activation='swish'),
    tf.keras.layers.Dense(4, activation='softmax')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X_train, y_train, epochs=50,batch_size =50, validation_split=0.2,callbacks=[early_stopping], verbose=1)

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy:', accuracy)

Epoch 1/50
164/164 [==============================] - 3s 6ms/step - loss: 284.3905 - accuracy: 0.5146 - val_loss: 99.9799 - val_accuracy: 0.4695
Epoch 2/50
164/164 [==============================] - 1s 5ms/step - loss: 62.6378 - accuracy: 0.5878 - val_loss: 30.0503 - val_accuracy: 0.6268
Epoch 3/50
164/164 [==============================] - 1s 5ms/step - loss: 58.7347 - accuracy: 0.5895 - val_loss: 80.0037 - val_accuracy: 0.5085
Epoch 4/50
164/164 [==============================] - 1s 5ms/step - loss: 54.3453 - accuracy: 0.5929 - val_loss: 26.1641 - val_accuracy: 0.6957
Epoch 5/50
164/164 [==============================] - 1s 5ms/step - loss: 41.7254 - accuracy: 0.6154 - val_loss: 40.2235 - val_accuracy: 0.4885
Epoch 6/50
164/164 [==============================] - 1s 5ms/step - loss: 34.5830 - accuracy: 0.6027 - val_loss: 91.8342 - val_accuracy: 0.1045
Epoch 7/50
164/164 [==============================] - 1s 5ms/step - loss: 29.9820 - accuracy: 0.6187 - val_loss: 17.1697 - val_accuracy

In [24]:
column_array = ['number of bedrooms','number of bathrooms','living area','lot area','number of floors','waterfront present','number of views','condition of the house','grade of the house','Area of the house(excluding basement)','Area of the basement','Built Year','Renovation Year','Postal Code','living_area_renov','lot_area_renov','Number of schools nearby','Distance from the airport']
df = pd.DataFrame(X_test,columns = column_array)
df.head()


,number of bedrooms,number of bathrooms,living area,lot area,number of floors,waterfront present,number of views,condition of the house,grade of the house,Area of the house(excluding basement),Area of the basement,Built Year,Renovation Year,Postal Code,living_area_renov,lot_area_renov,Number of schools nearby,Distance from the airport
0,1,2,1170,1456,2,0,0,3,8,1070,100,2007,0,122064,1360,1730,2,74
1,3,2,1840,9900,1,0,0,3,9,1840,0,1978,0,122030,2730,9900,3,56
2,2,1,1540,6872,1,0,0,4,7,820,720,1946,0,122003,1420,5538,3,72
3,5,2,2090,8712,1,0,0,3,8,1420,670,1966,0,122029,2490,8712,1,69
4,3,3,1670,1350,3,0,0,3,9,1350,320,2014,0,122009,1520,4800,3,59


**TESTING WITH A RANDOM ROW IN THE TEST DATASET**

In [25]:
output_dict = {0:"0-500k",1:"500k-1M",2:"1M-1.5M",3:"1.5M+"}
input_row = df.iloc[90].values.reshape(1, 18)
pred = model.predict(input_row)
arr = np.array(pred)
max_index = arr.argmax()
act_val = (np.array(y_test[5]).argmax())
print("PREDICTED VALUE:",output_dict[max_index])
print("ACTUAL VALUE:",output_dict[act_val])

1/1 [==============================] - 0s 126ms/step
PREDICTED VALUE: 0-500k
ACTUAL VALUE: 0-500k


In [30]:
input_row = df.iloc[45].values.reshape(1, 18)
pred = model.predict(input_row)
arr = np.array(pred)
max_index = arr.argmax()
act_val = (np.array(y_test[200]).argmax())
print("PREDICTED VALUE:",output_dict[max_index])
print("ACTUAL VALUE:",output_dict[act_val])

1/1 [==============================] - 0s 20ms/step
PREDICTED VALUE: 500k-1M
ACTUAL VALUE: 500k-1M


**ACTUAL VALUE VS PREDICTED VALUE (TABULAR FORM FOR FIRST 250 ROWS OF TEST DATASET)**

In [31]:
pred_values = []
actual_values = []
y_prd = model.predict(df)
for i in range(len(y_prd)):
  pred_values.append(output_dict[np.array(y_prd[i]).argmax()])
for i in range(len(y_test)):
  actual_val = np.array(y_test[i]).argmax()
  actual_values.append(output_dict[actual_val])
actual_values = np.array(actual_values)
actual_val_df = pd.DataFrame(actual_values,columns = ["ACTUAL VALUE"])
pred_values = np.array(pred_values)
pred_val_df = pd.DataFrame(pred_values,columns = ["PREDICTED VALUE"])

merged_df = pd.concat([actual_val_df, pred_val_df], axis=1)
merged_df['IS CORRECT?'] = merged_df['ACTUAL VALUE'] == merged_df['PREDICTED VALUE']


138/138 [==============================] - 0s 2ms/step


In [28]:
merged_df.head(45)

,ACTUAL VALUE,PREDICTED VALUE,IS CORRECT?
0,0-500k,0-500k,True
1,500k-1M,500k-1M,True
2,500k-1M,0-500k,False
3,500k-1M,500k-1M,True
4,500k-1M,0-500k,False
5,0-500k,0-500k,True
6,500k-1M,500k-1M,True
7,500k-1M,0-500k,False
8,1M-1.5M,500k-1M,False
9,500k-1M,500k-1M,True


In [29]:
corr_count = (merged_df["IS CORRECT?"] == True).sum()
acc = corr_count/merged_df.shape[0]
print("Accuracy:{}%".format(acc*100))

Accuracy:67.92065663474692%
